# Close approach

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [28]:
import numpy as np
from numpy.linalg import norm

# import plotting modules
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib.widgets as mw  # get access to the widgets

%matplotlib qt

from galaxy.db import DB
from galaxy.galaxies import Galaxies
from galaxy.galaxy import Galaxy
from galaxy.timecourse import TimeCourse
from galaxy.plots import Plots
from galaxy.approaches import Approaches

In [2]:
import mpl_scatter_density

# Make the norm object to define the image stretch
from astropy.visualization import LogStretch
from astropy.visualization.mpl_normalize import ImageNormalize
lognorm = ImageNormalize(vmin=0., vmax=1000, stretch=LogStretch())

In [3]:
#adjust tick label font size
label_size = 14
matplotlib.rcParams['xtick.labelsize'] = label_size 
matplotlib.rcParams['ytick.labelsize'] = label_size

In [29]:
def plot_density_views(disks, title, xlim=150, ylim=150, pngout=False, fname=None):
    fig = plt.figure(figsize=(10,10))
    fontsize = 16

    # top left
    ax0 = fig.add_subplot(1, 1, 1, projection='scatter_density')
    ax0.scatter_density(disks[0], disks[1], norm=lognorm)

    ax0.set_xlim(-xlim, xlim)
    ax0.set_ylim(-ylim, ylim)

    # Add axis labels (standard pyplot)
    ax0.set_xlabel('x (kpc)', fontsize=fontsize)
    ax0.set_ylabel('y (kpc)', fontsize=fontsize)
    
    # Save file
    if pngout:
        plt.savefig(fname, dpi='figure')
        plt.close();  

In [6]:
snap = 301
disks, t = get_xyz(snap)
title = f"\n\nSnap: {snap:03}\nElapsed time: {t:5.3f} Gyr"

In [7]:
plot_density_views(disks, title, pngout=False, fname=f"approach_{snap:03}.png")
print(snap, end=' ')

301 

/home/colin/.pyenv/versions/miniconda3-4.2.12/envs/400b/lib/python3.6/site-packages/mpl_scatter_density/generic_density_artist.py:74: RuntimeWarning: All-NaN slice encountered
  vmin = self._density_vmin(array)
/home/colin/.pyenv/versions/miniconda3-4.2.12/envs/400b/lib/python3.6/site-packages/mpl_scatter_density/generic_density_artist.py:79: RuntimeWarning: All-NaN slice encountered
  vmax = self._density_vmax(array)


In [62]:
snap = 300
app = Approaches(snap=snap, usesql=True)
t = app.time.value / 1000

In [64]:
# get two (3,N) arrays with just position/velocity coordinates
xyz = app.xyz()
vxyz = app.vxyz()

v = norm(vxyz, axis=0)

# center the collection visually (not CoM)
centroid = np.mean(xyz, axis=1)
xyz -= centroid[:,np.newaxis]
xyz.shape, v.shape

((3, 1166500), (1166500,))

In [87]:
index = None

def callbackRectangle( click, release ): # the events are click and release
    """
    """
    
    global index
    
#     extent = [ click.xdata, release.xdata, click.ydata, release.ydata ]
#     print( f"box extent is {extent}") 
    
    # create a rectangle
    width = np.abs(release.xdata - click.xdata)
    height = np.abs(release.ydata - click.ydata)
    
    rect = plt.Rectangle( (click.xdata, click.ydata), width, height,
                            fill=False, color='yellow', lw=1)
    ax0.add_patch(rect)
    
    xmin = min([click.xdata, release.xdata])
    xmax = max([click.xdata, release.xdata])
    ymin = min([click.ydata, release.ydata])
    ymax = max([click.ydata, release.ydata])
    
    index = np.where( (x > xmin) & (x < xmax) & (y > ymin) & (y < ymax) )
    
    add_velocity_plot()
    
#     ref = ax[1].scatter(x[index], y[index], s=1)

    # Save the file

In [88]:
def onKeyPressed(event):
    
    # eventuallywant to reset by removing selection
    pass

#     if event.key in ['R', 'r']:
#         ax.set_xlim(-30,30)
#         ax.set_ylim(-30,30)

In [106]:
def add_velocity_plot():
    ax1 = fig.add_subplot(1, 3, 2)

    if index is not None:
        ax1.scatter(x[index], y[index], c=v[index], cmap='magma', s=3)
#     plt.colorbar()

    # Add axis labels (standard pyplot)
    ax1.set_xlabel('x (kpc)', fontsize=fontsize)
    ax1.set_ylabel('y (kpc)', fontsize=fontsize)


In [107]:
x,y,z = xyz

fig = plt.figure(figsize=(18,6))
fontsize = 16

xlim = ylim = 150

# top left
ax0 = fig.add_subplot(1, 3, 1, projection='scatter_density')
ax0.scatter_density(x, y, norm=lognorm)

ax0.set_xlim(-xlim, xlim)
ax0.set_ylim(-ylim, ylim)

# Add axis labels (standard pyplot)
ax0.set_xlabel('x (kpc)', fontsize=fontsize)
ax0.set_ylabel('y (kpc)', fontsize=fontsize)




rs = mw.RectangleSelector( ax0,                        # the axes to attach to
                           callbackRectangle,         # the callback function
                           drawtype='box',            # draw a box when selecting a region
                           button=[1, 3],             # allow us to use left or right mouse button
                                                      #button 1 is left mouse button
                           minspanx=5, minspany=5,    # don't accept a box of fewer than 5 pixels
                           spancoords='pixels' )      # units for above

# to detect the 'R' key press to reset the image
plt.connect("key_press_event", onKeyPressed);

/home/colin/.pyenv/versions/miniconda3-4.2.12/envs/400b/lib/python3.6/site-packages/ipykernel_launcher.py:2: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  


In [78]:
sel_inx = index[0]

selected = app.data[sel_inx]
print(len(sel_inx))
selected[:4]

281


array([('M31', 2, 2.e-05, -23.9505, 173.416, -81.1963,  94.7691, -163.578, -11.0373 ),
       ('M31', 2, 2.e-05, -28.3972, 182.507, -80.4641, 117.235 , -125.089, -39.2443 ),
       ('M31', 2, 2.e-05, -25.0362, 170.492, -79.321 , 138.061 , -182.646,  -3.75729),
       ('M31', 2, 2.e-05, -24.6503, 171.797, -79.4012, 104.626 , -183.118,  19.3082 )],
      dtype=[('galname', '<U3'), ('type', 'u1'), ('m', '<f4'), ('x', '<f4'), ('y', '<f4'), ('z', '<f4'), ('vx', '<f4'), ('vy', '<f4'), ('vz', '<f4')])